In [1]:
!nvidia-smi

Sat Aug 26 07:06:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -U -qq gdown kaggle

In [3]:
from google.colab import drive
drive.mount('/content/drive')

import os
!mkdir -p ~/.kaggle/ && cp /content/drive/MyDrive/all_AI_code/kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Mounted at /content/drive


In [4]:
!kaggle competitions download -c fruit-ripening

100% 2.18G/2.18G [02:16<00:00, 21.0MB/s]
100% 2.18G/2.18G [02:16<00:00, 17.1MB/s]


In [5]:
import zipfile
import random
from pathlib import Path
from glob import glob

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [6]:
with zipfile.ZipFile("./fruit-ripening.zip", "r") as zip_ref:
    print("Unzipping main_zip data...")
    zip_ref.extractall('/content/data/')

os.remove("./fruit-ripening.zip")

Unzipping main_zip data...


In [7]:
ls = glob('/content/data/train/train/*/*.png')
df_submit = pd.read_csv('/content/data/submission.csv')
df_submit['path'] = '/content/data/test/test/' + df_submit['FileName']
df_submit.head()

,FileName,Class,path
0,abdy9380.png,0.0,/content/data/test/test/abdy9380.png
1,adua0170.png,0.0,/content/data/test/test/adua0170.png
2,aebq0222.png,5.0,/content/data/test/test/aebq0222.png
3,aeew6117.png,NaN,/content/data/test/test/aeew6117.png
4,aegl6733.png,NaN,/content/data/test/test/aegl6733.png


In [8]:
df_data = pd.DataFrame(ls,columns = ['path'])
df_data['label'] = df_data['path'].map(lambda x:int(x.split('/')[-2]))
df_data.sample(5)

,path,label
2221,/content/data/train/train/6/IMG_20220426_22510...,6
1352,/content/data/train/train/0/IMG_20220427_17273...,0
879,/content/data/train/train/0/IMG_20220427_17052...,0
1548,/content/data/train/train/0/IMG_20220427_15495...,0
545,/content/data/train/train/0/IMG_20220426_16450...,0


In [9]:
df_data['label'].value_counts()

0    2061
1     202
2     127
3      86
6      59
4      59
5      53
7      41
Name: label, dtype: int64

In [10]:
# Install Timm (Need to restart the runtime after finish install )
!pip -qq install git+https://github.com/rwightman/pytorch-image-models.git
!pip -qq install lightning transformers datasets evaluate pillow==9.2.0

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 93.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.7/70.7 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.7/596.7 kB 42.2 M

In [12]:
import torch
import torch.nn as nn
import torchvision.transforms as T
from torch.utils.data import DataLoader, random_split, Dataset
from torch.utils.tensorboard import SummaryWriter

# Pytorch Image model (TIMM) library: a library for state-of-the-art image classification
import timm
import timm.optim
import timm.scheduler
from timm.data import ImageDataset, create_dataset, create_loader
from timm.data.transforms_factory import create_transform

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from PIL import Image
import cv2
import evaluate

import numpy as np
import pandas as pd
from scipy import stats

from tqdm.notebook import tqdm

from glob import glob

from sklearn.model_selection import StratifiedKFold,KFold

from lightning.fabric import Fabric

from copy import copy
import time
import shutil

from sklearn.utils.class_weight import compute_class_weight

In [54]:
# Transform image data based on ImageNet's mean and std
mean = torch.tensor([0.485, 0.456, 0.406])
std = torch.tensor([0.229, 0.224, 0.225])
data_transforms = {
    "train": T.Compose([
        # T.RandomHorizontalFlip(p=0.5),
        # T.RandomVerticalFlip(p=0.5),
        # T.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
        # T.RandomRotation(degrees=(-15, 15)),
        # T.RandomAutocontrast(),
        T.Resize((384, 384)),
        T.ToTensor(),
        # T.Normalize(mean=mean, std=std)
    ]),
    "valid": T.Compose([
        # T.RandomHorizontalFlip(p=0.5),
        # T.RandomRotation(degrees=(-10, 10)),
        T.Resize((384, 384)),
        T.ToTensor(),
        # T.Normalize(mean=mean, std=std)
    ]),
    "test": T.Compose([
        T.Resize((384, 384)),
        T.ToTensor(),
        # T.Normalize(mean=mean, std=std)
    ])
}

In [13]:
# # Use ImageFolder to create dataset(s)
# from torchvision import datasets
# dataset = datasets.ImageFolder(root=train_dir, # target folder of images
#                                   transform=transforms["train"], # transforms to perform on data (images)
#                                   target_transform=None) # transforms to perform on labels (if necessary)
# dataset.class_to_idx, len(dataset)

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [55]:
torch.set_float32_matmul_precision('high')
fabric = Fabric(accelerator="cuda") # , precision="16-mixed"
fabric.launch()

In [56]:
class trainDataset(Dataset):
    def __init__(self, df = df_data, transforms = data_transforms, mode='train', sample = 600):
        self.data_1 = df[df['label']==0].sample(sample)
        self.data_2 = df[df['label']!=0].sample(sample)
        self.data = pd.concat([self.data_1,self.data_2]).sample(frac=1).reset_index(drop=True)
        self.transforms = transforms[mode]
    def __len__(self):
        return len(self.data)

    def preprocess(self, img_path):
      img_rgb = cv2.imread(img_path)
      img = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
      mask = np.zeros(img.shape, dtype=np.uint8)
      _,th = cv2.threshold(img,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
      dl_img = cv2.dilate(th, kernel=np.ones((5, 5), np.uint8), iterations=1)
      dl_img[:,:150] = 0
      dl_img[:,650:] = 0
      dl_img = cv2.erode(dl_img, kernel=np.ones((3, 3), np.uint8), iterations=1)
      contours, hierarchy = cv2.findContours(dl_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
      max_c = max(contours, key = cv2.contourArea)
      mask = cv2.fillPoly(mask, pts=[max_c], color=255)
      final = cv2.bitwise_and(img_rgb, img_rgb, mask = mask)
      destRGB = cv2.cvtColor(final, cv2.COLOR_BGR2RGB)
      return destRGB[100:500,200:600]

    def __getitem__(self, index):
        path = self.data['path'][index]
        label = int(path.split('/')[-2])
        img = self.preprocess(path)
        img = Image.fromarray(img)
        img = self.transforms(img)
        return img, label

In [23]:
!pip -q install GPUtil

  Preparing metadata (setup.py) ... done


In [52]:
import GPUtil

def free_gpu_cache():
    print("Initial GPU Usage")
    GPUtil.showUtilization()
    torch.cuda.empty_cache()
    print("GPU Usage after emptying the cache")
    GPUtil.showUtilization()

In [57]:
free_gpu_cache()

Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 19% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 |  0% | 19% |


In [58]:
timm.list_models('*efficientnet*', pretrained=True)

['efficientnet_b0.ra_in1k',
 'efficientnet_b1.ft_in1k',
 'efficientnet_b1_pruned.in1k',
 'efficientnet_b2.ra_in1k',
 'efficientnet_b2_pruned.in1k',
 'efficientnet_b3.ra2_in1k',
 'efficientnet_b3_pruned.in1k',
 'efficientnet_b4.ra2_in1k',
 'efficientnet_b5.sw_in12k',
 'efficientnet_b5.sw_in12k_ft_in1k',
 'efficientnet_el.ra_in1k',
 'efficientnet_el_pruned.in1k',
 'efficientnet_em.ra2_in1k',
 'efficientnet_es.ra_in1k',
 'efficientnet_es_pruned.in1k',
 'efficientnet_lite0.ra_in1k',
 'efficientnetv2_rw_m.agc_in1k',
 'efficientnetv2_rw_s.ra2_in1k',
 'efficientnetv2_rw_t.ra2_in1k',
 'gc_efficientnetv2_rw_t.agc_in1k',
 'tf_efficientnet_b0.aa_in1k',
 'tf_efficientnet_b0.ap_in1k',
 'tf_efficientnet_b0.in1k',
 'tf_efficientnet_b0.ns_jft_in1k',
 'tf_efficientnet_b1.aa_in1k',
 'tf_efficientnet_b1.ap_in1k',
 'tf_efficientnet_b1.in1k',
 'tf_efficientnet_b1.ns_jft_in1k',
 'tf_efficientnet_b2.aa_in1k',
 'tf_efficientnet_b2.ap_in1k',
 'tf_efficientnet_b2.in1k',
 'tf_efficientnet_b2.ns_jft_in1k',
 'tf_e

In [59]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [60]:
all_eval_scores = []
writer = SummaryWriter()

# Cross Validation Configuration
metric = evaluate.load("f1")
train_batch_size = 8
eval_batch_size = 8
num_accumulate = 2
num_epochs = 10

# Load Model
model_name = "hf_hub:timm/tf_efficientnet_b7.ns_jft_in1k"
model = timm.create_model(model_name, pretrained=True, num_classes=8)

# Load Optimizer and Scheduler
criterion = nn.CrossEntropyLoss()
optimizer = timm.optim.create_optimizer_v2(model, opt="AdamW", lr=5e-4)
# optimizer = timm.optim.Lookahead(optimizer, alpha=0.5, k=6)    # update the slow weight every k steps
                                                                # update the optimizer by combine slow weight and fast weight * alpha

model, optimizer = fabric.setup(model, optimizer)

scheduler = timm.scheduler.create_scheduler_v2(optimizer, num_epochs=num_epochs)[0]

info = {
        "metric_train": [],
        "metric_val": [],
        "train_loss": [],
        "val_loss": [],
        "best_metric_val": -999,
        "best_val_loss": 999,
    }
for epoch in range(num_epochs):
    train_loss_epoch = []
    val_loss_epoch = []

    train_preds = []
    train_targets = []

    val_preds = []
    val_targets = []

    train_dataset = trainDataset(df_data,transforms = data_transforms, mode = 'train', sample = 600)
    valid_dataset = trainDataset(df_data,transforms = data_transforms, mode = 'valid', sample = 400)

    train_dataloader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True, num_workers=2, pin_memory=True)
    val_dataloader = DataLoader(valid_dataset, batch_size=eval_batch_size, shuffle=False, num_workers=2, pin_memory=True)


    train_dataloader = fabric.setup_dataloaders(train_dataloader)
    val_dataloader = fabric.setup_dataloaders(val_dataloader)
    num_updates = epoch * len(train_dataloader)

    ### === Train Loop === ###
    ## Time
    start_time = time.monotonic()

    model.train()
    for idx, batch in enumerate(tqdm(train_dataloader, desc=f"Epoch: {epoch+1} | Training ", leave=False)):
        inputs, targets = batch
        # inputs = {k: v.to(device) for k,v in inputs.items()}
        outputs = model(inputs.to(device))
        loss = criterion(outputs, targets.to(device))
        writer.add_scalar("Loss/train", loss, epoch)

        fabric.backward(loss)

        # === Gradient Accumulation === #
        if ((idx + 1) % num_accumulate == 0) or (idx + 1 == len(train_dataloader)):
            optimizer.step()
            scheduler.step_update(num_updates=num_updates)
            optimizer.zero_grad()
        # ============================= #

        train_loss_epoch.append(loss.item())
        train_preds += outputs.argmax(-1).detach().cpu().tolist()
        train_targets += targets.tolist()
    ### ==================== ###

    # optimizer.sync_lookahead()              # Sync slow weight and fast weight
    scheduler.step(epoch + 1)

    ### === Evaluation Loop === ###
    model.eval()
    with torch.no_grad():
        for batch in tqdm(val_dataloader, desc=f"Epoch: {epoch+1} | Evaluating ", leave=False):
            inputs, targets = batch
            # inputs = {k: v.to(device) for k,v in inputs.items()}
            outputs = model(inputs.to(device))
            loss = criterion(outputs, targets.to(device))
            writer.add_scalar("Loss/vaild", loss, epoch)

            # Log Values
            val_loss_epoch.append(loss.item())
            val_preds += outputs.argmax(-1).detach().cpu().tolist()
            val_targets += targets.tolist()

    ### ======================= ###
    ## Time train finish
    end_time = time.monotonic()

    # Log Data
    metric_train = metric.compute(predictions=train_preds, references=train_targets, average='macro')['f1']
    metric_val = metric.compute(predictions=val_preds, references=val_targets, average='macro')['f1']

    info["metric_train"].append(metric_train)
    info["metric_val"].append(metric_val)

    info["train_loss"].append(np.average(train_loss_epoch))
    info["val_loss"].append(np.average(val_loss_epoch))

    if metric_val > info["best_metric_val"]:
    # if info["val_loss"][-1] < info["best_val_loss"]:
        # print("New Best Score!")
        # print("New Best Val Loss")
        info["best_metric_val"] = metric_val
        # info["best_val_loss"] = info["val_loss"][-1]
        torch.save(model, f"/content/tf_efficientnet_b7_best_f1.pt")

    writer.flush()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    # print(f"Using time of Epoch: {epoch+1} | {epoch_mins}m {epoch_secs}s")
    print(f"Epoch: {epoch+1} | {epoch_mins}m {epoch_secs}s | Loss : train {np.average(train_loss_epoch):.4f} valid {np.average(val_loss_epoch):.4f} | F1 : train {metric_train*100:.2f}% valid {metric_val*100:.2f}%")
    # print(info)
    # save all best metric val
    # all_eval_scores.append(info["best_metric_val"])
writer.close()

Training :   0%|          | 0/150 [00:00<?, ?it/s]

Evaluating :   0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 1 : 2m 25s | Loss (train,valid) : 1.154 0.469 | F1 (train,valid) : 44.79% 75.22%


Training :   0%|          | 0/150 [00:00<?, ?it/s]

Evaluating :   0%|          | 0/100 [00:00<?, ?it/s]

...

Evaluating :   0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 9 : 2m 22s | Loss (train,valid) : 0.009 0.006 | F1 (train,valid) : 99.83% 99.93%


Training :   0%|          | 0/150 [00:00<?, ?it/s]

Evaluating :   0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 10 : 2m 23s | Loss (train,valid) : 0.010 0.001 | F1 (train,valid) : 99.87% 100.00%


In [35]:
torch.save(model, f"/content/tf_efficientnet_b7_ep30.pt")

In [36]:
fake_test_dataset = trainDataset(df_data,transforms = data_transforms, mode = 'test', sample = 600)
fake_test_dataloader = DataLoader(fake_test_dataset, batch_size=eval_batch_size, shuffle=False, num_workers=2, pin_memory=True)
loaded_model = torch.load(f"/content/tf_efficientnet_b7_best_f1.pt")
ls_ans_fake_test = []
ls_target =[]

loaded_model.eval()
with torch.no_grad():
    for batch in tqdm(fake_test_dataloader, desc="Evaluating ", leave=False):
        inputs, targets = batch
        outputs = loaded_model(inputs.to(device))
        y_pred = torch.argmax(torch.softmax(outputs, dim=1), dim=1)
        ls_ans_fake_test.extend(y_pred.tolist())
        ls_target.extend(targets.tolist())

Evaluating :   0%|          | 0/150 [00:00<?, ?it/s]

In [37]:
print(classification_report(ls_ans_fake_test, ls_target))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       600
           1       1.00      1.00      1.00       194
           2       1.00      1.00      1.00       124
           3       1.00      1.00      1.00        80
           4       1.00      1.00      1.00        57
           5       1.00      1.00      1.00        51
           6       1.00      1.00      1.00        55
           7       1.00      1.00      1.00        39

    accuracy                           1.00      1200
   macro avg       1.00      1.00      1.00      1200
weighted avg       1.00      1.00      1.00      1200



In [38]:
fake_test_dataset = trainDataset(df_data,transforms = data_transforms, mode = 'test', sample = 600)
fake_test_dataloader = DataLoader(fake_test_dataset, batch_size=eval_batch_size, shuffle=False, num_workers=2, pin_memory=True)
loaded_model = torch.load(f"/content/tf_efficientnet_b7_ep30.pt")
ls_ans_fake_test = []
ls_target =[]

loaded_model.eval()
with torch.no_grad():
    for batch in tqdm(fake_test_dataloader, desc="Evaluating ", leave=False):
        inputs, targets = batch
        outputs = loaded_model(inputs.to(device))
        y_pred = torch.argmax(torch.softmax(outputs, dim=1), dim=1)
        ls_ans_fake_test.extend(y_pred.tolist())
        ls_target.extend(targets.tolist())
print(classification_report(ls_ans_fake_test, ls_target))

Evaluating :   0%|          | 0/150 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       599
           1       1.00      0.99      1.00       193
           2       1.00      1.00      1.00       122
           3       1.00      1.00      1.00        83
           4       1.00      1.00      1.00        54
           5       1.00      1.00      1.00        51
           6       1.00      1.00      1.00        58
           7       1.00      1.00      1.00        40

    accuracy                           1.00      1200
   macro avg       1.00      1.00      1.00      1200
weighted avg       1.00      1.00      1.00      1200



In [61]:
# No augment tf_efficientnet_b7_best_f1
fake_test_dataset = trainDataset(df_data,transforms = data_transforms, mode = 'test', sample = 600)
fake_test_dataloader = DataLoader(fake_test_dataset, batch_size=eval_batch_size, shuffle=False, num_workers=2, pin_memory=True)
loaded_model = torch.load(f"/content/tf_efficientnet_b7_best_f1.pt")
ls_ans_fake_test = []
ls_target =[]

loaded_model.eval()
with torch.no_grad():
    for batch in tqdm(fake_test_dataloader, desc="Evaluating ", leave=False):
        inputs, targets = batch
        outputs = loaded_model(inputs.to(device))
        y_pred = torch.argmax(torch.softmax(outputs, dim=1), dim=1)
        ls_ans_fake_test.extend(y_pred.tolist())
        ls_target.extend(targets.tolist())
print(classification_report(ls_ans_fake_test, ls_target))

Evaluating :   0%|          | 0/150 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       597
           1       1.00      0.98      0.99       197
           2       1.00      1.00      1.00       122
           3       1.00      1.00      1.00        81
           4       1.00      1.00      1.00        56
           5       1.00      1.00      1.00        50
           6       1.00      1.00      1.00        56
           7       1.00      1.00      1.00        41

    accuracy                           1.00      1200
   macro avg       1.00      1.00      1.00      1200
weighted avg       1.00      1.00      1.00      1200



In [62]:
class testDataset(Dataset):
    def __init__(self, df = df_data, transforms = data_transforms, mode = 'test'):
        self.data = df
        self.transforms = transforms[mode]

    def __len__(self):
        return len(self.data)

    def preprocess(self, img_path):
      img_rgb = cv2.imread(img_path)
      img = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
      mask = np.zeros(img.shape, dtype=np.uint8)
      _,th = cv2.threshold(img,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
      dl_img = cv2.dilate(th, kernel=np.ones((5, 5), np.uint8), iterations=1)
      dl_img[:,:150] = 0
      dl_img[:,650:] = 0
      dl_img = cv2.erode(dl_img, kernel=np.ones((3, 3), np.uint8), iterations=1)
      contours, hierarchy = cv2.findContours(dl_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
      max_c = max(contours, key = cv2.contourArea)
      mask = cv2.fillPoly(mask, pts=[max_c], color=255)
      final = cv2.bitwise_and(img_rgb, img_rgb, mask = mask)
      destRGB = cv2.cvtColor(final, cv2.COLOR_BGR2RGB)
      return destRGB[100:500,200:600]

    def __getitem__(self, index):
        path = self.data['path'][index]
        img = self.preprocess(path)
        img = Image.fromarray(img)
        img = self.transforms(img)
        return img

test_dataset = testDataset(df_submit, transforms=data_transforms, mode='test')
test_dataloader = DataLoader(test_dataset, batch_size=eval_batch_size, shuffle=False, num_workers=2, pin_memory=True)

In [63]:
ls_ans = []

loaded_model = torch.load(f"/content/tf_efficientnet_b7_best_f1.pt")
loaded_model.eval()
with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Evaluating ", leave=False):
        inputs = batch
        outputs = loaded_model(inputs.to(device))
        y_pred = torch.argmax(torch.softmax(outputs, dim=1), dim=1)
        ls_ans.extend(y_pred.tolist())

Evaluating :   0%|          | 0/85 [00:00<?, ?it/s]

In [64]:
ls_ans[:5]

[0, 0, 5, 0, 0]

In [65]:
df_submit['Class'][3:] = ls_ans[3:]
df_submit['Class'] = df_submit['Class'].map(int)
df_submit

<ipython-input-65-3d8e79339bbb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_submit['Class'][3:] = ls_ans[3:]


,FileName,Class,path
0,abdy9380.png,0,/content/data/test/test/abdy9380.png
1,adua0170.png,0,/content/data/test/test/adua0170.png
2,aebq0222.png,5,/content/data/test/test/aebq0222.png
3,aeew6117.png,0,/content/data/test/test/aeew6117.png
4,aegl6733.png,0,/content/data/test/test/aegl6733.png
...,...,...,...
668,zvxc1988.png,5,/content/data/test/test/zvxc1988.png
669,zyms7894.png,0,/content/data/test/test/zyms7894.png
670,zzcb3748.png,0,/content/data/test/test/zzcb3748.png
671,zzts1908.png,2,/content/data/test/test/zzts1908.png


In [43]:
# tf_efficientnet_b7_ep30
df_submit['Class'].value_counts()

0    540
1     40
2     28
5     21
3     14
7     12
6     11
4      7
Name: Class, dtype: int64

In [49]:
# tf_efficientnet_b7_best_f1
df_submit['Class'].value_counts()

0    542
1     41
2     26
5     20
7     13
6     12
3     12
4      7
Name: Class, dtype: int64

In [66]:
# No augment tf_efficientnet_b7_best_f1
df_submit['Class'].value_counts()

0    548
2     38
1     27
5     22
7     14
6     10
3      8
4      6
Name: Class, dtype: int64

In [67]:
df_submit[['FileName','Class']].to_csv('tf_efficientnet_b7_best_f1_No_aug.csv', index=False)

In [68]:
!kaggle competitions submit -c fruit-ripening -f tf_efficientnet_b7_best_f1_No_aug.csv -m "tf_efficientnet_b7_best_f1_No_aug"

100% 9.87k/9.87k [00:02<00:00, 4.36kB/s]
Successfully submitted to Hackathon Online: Image Processing